# Langfuse(Self-hosted)の機能を最低限試すためのノートブック

以下の機能が対象です。

- Tracing - Low-level SDK, Decorator, LangChain integration
- Prompt Management
- Datasets
- Scores

また、事前に以下のようにLangfuseを起動していることを前提とします。

```sh
# ~/llm-observability
docker compose up -d
```

確認

```sh
docker compose ps
NAME                                  IMAGE                 COMMAND                  SERVICE           CREATED      STATUS                PORTS
llm-observability-db-1                postgres              "docker-entrypoint.s…"   db                7 days ago   Up 7 days (healthy)   0.0.0.0:5432->5432/tcp, :::5432->5432/tcp
llm-observability-langfuse-server-1   langfuse/langfuse:2   "dumb-init -- ./web/…"   langfuse-server   7 days ago   Up 7 days             0.0.0.0:3000->3000/tcp, :::3000->3000/tcp
```

## Langfuseの初期設定

適当にやってくれ

## 事前準備

In [ ]:
%pip install -r ../requirements.txt

必要な情報を環境変数（`.env`）から取得します

In [3]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())

endpoint = os.getenv("ENDPOINT")
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

Langfuseのクライアントを初期化します

In [4]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

## トレース

In [ ]:
from langchain_cohere.chat_models import ChatCohere

cohere_api_key = os.getenv("COHERE_API_KEY")

chat = ChatCohere(
    model="command-r-plus",
    cohere_api_key=cohere_api_key
)

Low-level SDK

In [6]:
import uuid

trace = langfuse.trace(
    name="bbql-chatbot-app",
    version="0.0.1-SNAPSHOT",
    tags=["staging"],
    session_id=str(uuid.uuid4())
)

In [ ]:
input = [{"role": "user", "content": "カルビクッパの作り方を教えてください"}]

generation = trace.generation(
    name="generation",
    model="command-r-plus",
    input=input
)

chat_completion = chat.invoke(
    input=input
)

output = chat_completion.content

generation.end(
    output=output
)

trace.update(input=input, output=output)

LangChainを使う場合は、LangChainのCallback用のHandlerがLangfuseから提供されているので、これを使うとチェーン全体のトレースが取得できます。

In [8]:
from langfuse.callback import CallbackHandler
import uuid

langchain_callback = CallbackHandler(
    host=endpoint,
    public_key=public_key,
    secret_key=secret_key,
    session_id=str(uuid.uuid4())
)

RAGの構成を作ります。詳細は、[setup-bbql-app.ipynb](./setup-bbql-app.ipynb)を参照してください。

In [9]:
from langchain_cohere import CohereEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import glob
from langchain.document_loaders import TextLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

cohere_api_key = os.getenv("COHERE_API_KEY")

embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-multilingual-v3.0"
)

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

files = glob.glob("../app/docs/*.txt")
documents = []

for file in files:
    loader = TextLoader(file_path=file)
    document = loader.load()
    documents.extend(document)

vector_store.add_documents(documents=documents)

retriever = vector_store.as_retriever()

prompt_template = PromptTemplate.from_template(template="""
以下のコンテキストに基づいて質問に対する回答を作成してください。

## コンテキスト

{context}

## 質問

{question}
""")

chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

In [ ]:
response = chain.stream(
    "カレーの作り方を教えてください",
    config={"callbacks": [langchain_callback]}
)

for chunk in response:
    print(chunk, end="")

LangChain のチェーン実行に対して、自動的にトレース情報が取得されていることが確認できます。

<img src="../images/langfuse-trace.png" width="50%">

## プロンプト管理

Langfuseでは、LLMアプリケーションが使うプロンプトをLangfuseとして構成管理することができます

プロンプトを作成します（UIでも実施可能です）

In [ ]:
prompt = """
以下のコンテキストに基づいて質問に対する回答を作成してください。

## コンテキスト
{{context}}

## 質問
{{question}}
"""

langfuse.create_prompt(
    name="bbql-app-prompt",
    prompt=prompt,
    labels=["production"],
    tags=["production"]
)

作成したプロンプトを読み込みます

In [ ]:
prompt = langfuse.get_prompt(
    name="bbql-app-prompt",
)

print(prompt.prompt)

変数のバインド

In [ ]:
compiled_prompt = prompt.compile(
    context="雰囲気で作れ！",
    question="カレーの作り方を教えてください"
)

print(compiled_prompt)

LangChainから使う場合は、`get_langchain_prompt`が使えるのでこちらを使うと良い

In [ ]:
langchain_prompt = prompt.get_langchain_prompt()

print(langchain_prompt)

RAGの中で使うならこんな感じ

In [ ]:
chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | PromptTemplate.from_template(langchain_prompt)
    | chat
    | StrOutputParser()
)

response = chain.stream(
    "カレーの作り方を教えてください",
    config={"callbacks": [langchain_callback]}
)

for chunk in response:
    print(chunk, end="")

プロンプトは、バージョン管理可能

In [ ]:
prompt2 = """
以下のコンテキストに基づいて質問に対する回答をBBっぽく作成してください。

## コンテキスト
{{context}}

## 質問
{{question}}
"""

# 同一名でプロンプトを作成すると、上書きされ Version が上がる
langfuse.create_prompt(
    name="bbql-app-prompt",
    prompt=prompt2,
    labels=["production"],
    tags=["production"]
)

Version: 1, Version: 2のプロンプトを参照してみる

In [ ]:
prompt_v1 = langfuse.get_prompt(
    name="bbql-app-prompt",
    version=1
)

prompt_v2 = langfuse.get_prompt(
    name="bbql-app-prompt",
    version=2
)

print(f"prompt_v1: {prompt_v1.prompt}")
print("*********************************")
print(f"prompt_v2: {prompt_v2.prompt}")

プロンプトをLLMアプリケーションのコード中に埋め込むと、プロンプトの変更 = LLMアプリケーションの再ビルドになるので結構手間となる。  
かつ、LLMアプリケーションはプロンプトだけ細かくチューニングしたいパターンが結構あるので、こういう機能が備わってくれるとありがたい

## Score

In [ ]:
query = "キノコを使ったおつまみとかないですかね"

response = chain.stream(
    query,
    config={"callbacks": [langchain_callback]}
)

for chunk in response:
    print(chunk, end="")

current_trace_id = langchain_callback.get_trace_id()

In [ ]:
langfuse.score(
    name="bbql-app-sample-feedback",
    value=1,
    data_type="NUMERIC",
    trace_id=current_trace_id,
    comment="求めていたものでした"
)

UIで目的のトレースを確認すると、評価がつけられていることが確認できる

<img src="../images/langfuse-score.png" width="50%"/>

上の例は、UI等で収集した人による評価の想定だが、LLM自身に評価をさせることもできる（いわゆる、LLM-as-a-Judge）  
ここでは、LangChainのEvaluatorを使用する

In [ ]:
from langchain.evaluation.loading import load_evaluator
from langchain.evaluation.schema import EvaluatorType

# Embedding(ベクトル空間に埋め込まれた文書間の類似性)に基づく評価を行う
evaluator = load_evaluator(
    evaluator=EvaluatorType.EMBEDDING_DISTANCE,
    embeddings=embeddings
)

# 評価対象のトレースを取得する
trace = langfuse.get_trace(id=current_trace_id)

# LangChainのEvaluatorを用いて評価をする
result = evaluator.evaluate_strings(
    prediction=trace.output,
    reference=query
)

# 評価に基づき、スコアとトレースを紐づける
langfuse.score(
    name="embedding-distance",
    value=result["score"],
    data_type="NUMERIC",
    trace_id=current_trace_id,
    comment="by embedding distance"
)

EvaluatorTypeを参照すると、使用可能なEvaluatorが数多く提供されていることが確認できる

```py
    QA = "qa"
    """Question answering evaluator, which grades answers to questions
    directly using an LLM."""
    COT_QA = "cot_qa"
    """Chain of thought question answering evaluator, which grades
    answers to questions using
    chain of thought 'reasoning'."""
    CONTEXT_QA = "context_qa"
    """Question answering evaluator that incorporates 'context' in the response."""
    PAIRWISE_STRING = "pairwise_string"
    """The pairwise string evaluator, which predicts the preferred prediction from
    between two models."""
    SCORE_STRING = "score_string"
    """The scored string evaluator, which gives a score between 1 and 10 
    to a prediction."""
    LABELED_PAIRWISE_STRING = "labeled_pairwise_string"
    """The labeled pairwise string evaluator, which predicts the preferred prediction
    from between two models based on a ground truth reference label."""
    LABELED_SCORE_STRING = "labeled_score_string"
    """The labeled scored string evaluator, which gives a score between 1 and 10
    to a prediction based on a ground truth reference label."""
    AGENT_TRAJECTORY = "trajectory"
    """The agent trajectory evaluator, which grades the agent's intermediate steps."""
    CRITERIA = "criteria"
    """The criteria evaluator, which evaluates a model based on a
    custom set of criteria without any reference labels."""
    LABELED_CRITERIA = "labeled_criteria"
    """The labeled criteria evaluator, which evaluates a model based on a
    custom set of criteria, with a reference label."""
    STRING_DISTANCE = "string_distance"
    """Compare predictions to a reference answer using string edit distances."""
    EXACT_MATCH = "exact_match"
    """Compare predictions to a reference answer using exact matching."""
    REGEX_MATCH = "regex_match"
    """Compare predictions to a reference answer using regular expressions."""
    PAIRWISE_STRING_DISTANCE = "pairwise_string_distance"
    """Compare predictions based on string edit distances."""
    EMBEDDING_DISTANCE = "embedding_distance"
    """Compare a prediction to a reference label using embedding distance."""
    PAIRWISE_EMBEDDING_DISTANCE = "pairwise_embedding_distance"
    """Compare two predictions using embedding distance."""
    JSON_VALIDITY = "json_validity"
    """Check if a prediction is valid JSON."""
    JSON_EQUALITY = "json_equality"
    """Check if a prediction is equal to a reference JSON."""
    JSON_EDIT_DISTANCE = "json_edit_distance"
    """Compute the edit distance between two JSON strings after canonicalization."""
    JSON_SCHEMA_VALIDATION = "json_schema_validation"
    """Check if a prediction is valid JSON according to a JSON schema."""
```